In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2ixegfj5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-2ixegfj5
  Resolved https://github.com/openai/whisper.git to commit 0a60fcaa9b86748389a656aa013c416030287d47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230918-py3-none-any.whl size=798399 sha256=65c2614fbac8cf451ce1ee3b76db85f4ec445c10d5da1586b0efed32dbf84d2c
  Stored in directory: /tmp/pip-ephem-wheel-cache-xvr2p_9q/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
import whisper
import os
import numpy as np
import torch

In [ ]:
torch.cuda.is_available()
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = whisper.load_model("base", device=device)
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 175MiB/s]


Model is multilingual and has 71,825,920 parameters.


In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
import pytube
import os
import sys

def youtube_preprocess(link):

  yt = pytube.YouTube(link)
  yt.register_on_progress_callback(show_progress_bar)

  destination = '.' #Alter your file download destination here

  #Fetching only the audio part of the video for faster download
  video = yt.streams.filter(only_audio = True).first()

  #Downloading and renaming as a wav file
  out_file = video.download(output_path=destination)
  base, ext = os.path.splitext(out_file)
  new_file = base + '.wav'
  os.rename(out_file, new_file)

  return new_file

# Display a download progress bar
def show_progress_bar(stream, _chunk, bytes_remaining):
  current = ((stream.filesize - bytes_remaining)/stream.filesize)
  percent = ('{0:.1f}').format(current*100)
  progress = int(50*current)
  status = '█' * progress + '-' * (50 - progress)
  sys.stdout.write(' ↳ |{bar}| {percent}%\r'.format(bar=status,
  percent=percent))

  sys.stdout.flush()

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
from pydub.utils import make_chunks
import math
import os

def chunk_by_size(my_file):
    myaudio = AudioSegment.from_file(my_file)
    channel_count = myaudio.channels    #Get channels
    sample_width = myaudio.sample_width #Get sample width
    duration_in_sec = len(myaudio)/1000 #Length of audio in seconds
    sample_rate = myaudio.frame_rate
    bit_depth = sample_width * 8 #Bit depth

    wav_file_size = (sample_rate * bit_depth * channel_count * duration_in_sec)/8

    file_split_size = 24000000 # 24Mb OR 24,000,000 bytes
    total_chunks =  wav_file_size // file_split_size

    '''
    Get chunk size by the following method:
    For X duration_in_sec (X) -->  Y wav_file_size
    For K duration in sec  (K) --> For 24MB file size
    K = X * 10Mb / Y
    '''

    chunk_length_in_sec = math.ceil((duration_in_sec * file_split_size)/wav_file_size)   #in seconds
    chunk_length_ms = chunk_length_in_sec * 1000
    chunks = make_chunks(myaudio, chunk_length_ms)

    os.mkdir("process_chunks")

    #Export all of the individual chunks as wav files
    for i, chunk in enumerate(chunks):
        chunk_name = "process_chunks/chunk{0}.wav".format(i)
        chunk.export(chunk_name, format="wav")
    return total_chunks

In [ ]:
!pip install youtube_preprocess

ERROR: Could not find a version that satisfies the requirement youtube_preprocess (from versions: none)
ERROR: No matching distribution found for youtube_preprocess


In [ ]:
# from videoproc import youtube_preprocess
# from chunking import chunk_by_size



audio_file = youtube_preprocess("https://www.youtube.com/watch?v=N1oNQcunK6o")
no_of_chunks = chunk_by_size(audio_file)

# for i in range(no_of_chunks):
#     curr_file = open(f"process_chunks/chunk{i}.wav", "rb")
#     # transcript = openai.Audio.translate("whisper-1", curr_file)

#     with open("videotext.txt","a") as f:
#         f.write(transcript["text"])

In [ ]:
audio = whisper.load_audio("/content/audio.wav")
#  ("/content/process_chunks/chunk0.wav")

audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)

In [ ]:
#detect language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: en


In [ ]:
#transcribing first 30 seconds
options = whisper.DecodingOptions(language="en", without_timestamps=True, fp16 = False)
result = whisper.decode(model, mel, options)
print(result.text)

Welcome to one little quarter. Facebook AI recently launched a very exciting model called No Language Left Behind NLLB. But the problem was there was no easy way to do inference and check. Usually a lot of companies whenever they release the model they either release or hugging face model hub or they would also attach a Google collab notebook where we can easily run that and see. But since this model was released NLLB


In [ ]:
# download the language model pretrained file
!wget https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin

--2023-10-02 07:19:29--  https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.34, 3.162.163.51, 3.162.163.11, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1176355829 (1.1G) [application/octet-stream]
Saving to: ‘lid218e.bin’

lid218e.bin         100%[===================>]   1.09G  17.4MB/s    in 60s     

2023-10-02 07:20:29 (18.8 MB/s) - ‘lid218e.bin’ saved [1176355829/1176355829]



In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199769 sha256=f71314fceef2fa25051acb22e518714a43fe5ac7b8ac184de6867e87e5508adc
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import fasttext

pretrained_lang_model = "/content/lid218e.bin" # path of pretrained model file
model_fasttext = fasttext.load_model(pretrained_lang_model)

In [ ]:
predictions = model_fasttext.predict(result.text, k=1)
print(predictions)

(('__label__eng_Latn',), array([1.00000286]))


In [ ]:
input_lang = predictions[0][0].replace('__label__', '')

In [ ]:
input_lang

'eng_Latn'

In [ ]:
!pip install -U pip transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.6 MB/s eta 0:00:00


In [ ]:
checkpoint = 'facebook/nllb-200-distilled-600M'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
target_lang = 'urd_Arab'
translation_pipeline = pipeline('translation',
                                model=model,
                                tokenizer=tokenizer,
                                src_lang=input_lang,
                                tgt_lang=target_lang,
                                max_length = 1024)
output = translation_pipeline(result.text)
print(result.text)
print(output[0]['translation_text'])
for i in result.text.lstrip("."):
  print(i)

Welcome to one little quarter. Facebook AI recently launched a very exciting model called No Language Left Behind NLLB. But the problem was there was no easy way to do inference and check. Usually a lot of companies whenever they release the model they either release or hugging face model hub or they would also attach a Google collab notebook where we can easily run that and see. But since this model was released NLLB
ایک چھوٹی سی سہ ماہی میں خوش آمدید۔ فیس بک اے آئی نے حال ہی میں ایک بہت ہی دلچسپ ماڈل لانچ کیا جسے No Language Left Behind NLLB کہا جاتا ہے۔ لیکن مسئلہ یہ تھا کہ نتیجہ اخذ کرنے اور جانچنے کا کوئی آسان طریقہ نہیں تھا۔ عام طور پر بہت سی کمپنیاں جب بھی ماڈل جاری کرتی ہیں تو وہ یا تو جاری کرتی ہیں یا چہرے کے ماڈل ہب کو گلے لگاتی ہیں یا وہ گوگل کولاب نوٹ بک بھی منسلک کرتی ہیں جہاں ہم آسانی سے اسے چلائیں اور دیکھیں گے۔ لیکن چونکہ یہ ماڈل جاری کیا گیا تھا NLLB


In [ ]:
 d = result.text.split(".")
 d

['Welcome to one little quarter',
 ' Facebook AI recently launched a very exciting model called No Language Left Behind NLLB',
 ' But the problem was there was no easy way to do inference and check',
 ' Usually a lot of companies whenever they release the model they either release or hugging face model hub or they would also attach a Google collab notebook where we can easily run that and see',
 ' But since this model was released NLLB']

In [ ]:
int(no_of_chunks)

5

In [ ]:
audio = whisper.load_audio("/content/audio.wav")
# open(f"/content/process_chunks/chunk0.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

AttributeError: ignored

In [ ]:
whisper_model = whisper.load_model("base", device=device)
print(
    f"Model is {'multilingual' if whisper_model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in whisper_model.parameters()):,} parameters."
)

Model is multilingual and has 71,825,920 parameters.


In [ ]:
for j in range(int(no_of_chunks)):
  path = "/content/process_chunks/chunk"+str(j)+".wav"
  audio =whisper.load_audio(path)
  # open(f"/content/process_chunks/chunk{j}.wav", "rb")
  audio = whisper.pad_or_trim(audio)
  mel = whisper.log_mel_spectrogram(audio).to(whisper_model.device)
  _, probs = whisper_model.detect_language(mel)
  print(f"Detected language: {max(probs, key=probs.get)}")
  #transcribing first 30 seconds
  options = whisper.DecodingOptions(language="en", fp16 = False)
  result = whisper.decode(whisper_model, mel, options)
  print(result.text)
  for i in result.text.split("."):
    output = translation_pipeline(i)
    print(i)
    print(output[0]['translation_text'])



Detected language: en
Welcome to one little quarter. Facebook AI recently launched a very exciting model called No Language Lift Behind NLLB. But the problem was there was no easy way to do inference and check. Usually a lot of companies whenever they release the model they either release or hugging face model hub or they would also attach a Google collab notebook where we can easily
Welcome to one little quarter
ایک چھوٹا سا ربع میں خوش آمدید
 Facebook AI recently launched a very exciting model called No Language Lift Behind NLLB
فیس بک اے آئی نے حال ہی میں ایک بہت ہی دلچسپ ماڈل لانچ کیا ہے جسے NLLB کے پیچھے کوئی زبان لفٹ نہیں کہا جاتا ہے
 But the problem was there was no easy way to do inference and check
لیکن مسئلہ یہ تھا کہ نتیجہ اخذ کرنے اور جانچنے کا کوئی آسان طریقہ نہیں تھا
 Usually a lot of companies whenever they release the model they either release or hugging face model hub or they would also attach a Google collab notebook where we can easily
عام طور پر بہت سی کمپنیاں جب بھ

In [ ]:
for i in d:
  output = translation_pipeline(i)
  print(i)
  print(output[0]['translation_text'])

Welcome to one little quarter
ایک چھوٹا سا ربع میں خوش آمدید
 Facebook AI recently launched a very exciting model called No Language Left Behind NLLB
فیس بک اے آئی نے حال ہی میں ایک بہت ہی دلچسپ ماڈل لانچ کیا ہے جسے No Language Left Behind NLLB کہا جاتا ہے
 But the problem was there was no easy way to do inference and check
لیکن مسئلہ یہ تھا کہ نتیجہ اخذ کرنے اور جانچنے کا کوئی آسان طریقہ نہیں تھا
 Usually a lot of companies whenever they release the model they either release or hugging face model hub or they would also attach a Google collab notebook where we can easily run that and see
عام طور پر بہت سی کمپنیاں جب بھی وہ ماڈل جاری کرتے ہیں وہ یا تو جاری یا چہرے ماڈل ہب کو گلے لگاتے ہیں یا وہ بھی ایک گوگل کلب نوٹ بک منسلک کریں گے جہاں ہم آسانی سے چل سکتے ہیں اور دیکھیں
 But since this model was released NLLB
لیکن چونکہ اس ماڈل کو جاری کیا گیا تھا NLLB


**timestamp**

In [ ]:
!pip install -U git+https://github.com/linto-ai/whisper-timestamped

  Cloning https://github.com/linto-ai/whisper-timestamped to /tmp/pip-req-build-8q6qcy_5
  Running command git clone --filter=blob:none --quiet https://github.com/linto-ai/whisper-timestamped /tmp/pip-req-build-8q6qcy_5
  Resolved https://github.com/linto-ai/whisper-timestamped to commit 127e8b738756420487210490d09938593244b3dd
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.5/645.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for whisper-timestamped: filename=whisper_timestamped-1.12.20-py3-none-any.whl size=44512 sha256=8530c2f0dd3b47e617caf7d2de614df572f58053ac41a07b96f682bd27be9c4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-2d8gf4tz/wheels/9d/fb/03/ae0b1c8b71edda71e33da77333c78117d879361bdd35de48e4
Successfully built whisper-timestamped


In [ ]:
import whisper_timestamped as whisper

audio = whisper.load_audio('/content/audio.wav')
model = whisper.load_model("base")

result = whisper.transcribe(model, audio, language="en")

import json
print(json.dumps(result, indent = 2, ensure_ascii = False))

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



100%|██████████| 81464/81464 [05:03<00:00, 267.98frames/s]


{
  "text": " Welcome to one little quarter. Facebook AI recently launched a very exciting model called No Language Lift Behind NLLB. But the problem was there was no easy way to do inference and check. Usually a lot of companies whenever they release the model they either release or hugging face model hub or they would also attach a Google collab notebook where we can easily run that and see. But since this model was released in LLB there was no support or there was no inference script that was easy available. You had to do a couple of steps to do that. But thanks to a very kind soul we don't have the name of the user so it's pluys. So pluys has actually created a helper script that will help you run Facebook's NLLB No language lift behind. So what I've done is I've taken that code. I've put together an instruction on Google collab and that is what we are going to see in this video where we are going to use Facebook's No Language Lift Behind model to do translation. Language translati

In [ ]:
for i, segment in enumerate(result['segments']):
  start, end = segment['start'], segment['end']
  print(i)
  print(f"00:00:{str(int(start)).replace('.', ',')} --> 00:00:{str(int(end)).replace('.', ',')}")
  print(segment['text'].strip())
  print()

0
00:00:0 --> 00:00:1
Welcome to one little quarter.

1
00:00:1 --> 00:00:10
Facebook AI recently launched a very exciting model called No Language Lift Behind NLLB.

2
00:00:10 --> 00:00:15
But the problem was there was no easy way to do inference and check.

3
00:00:16 --> 00:00:20
Usually a lot of companies whenever they release the model they either release or hugging

4
00:00:20 --> 00:00:24
face model hub or they would also attach a Google collab notebook where we can easily

5
00:00:25 --> 00:00:25
run that and see.

6
00:00:26 --> 00:00:33
But since this model was released in LLB there was no support or there was no inference

7
00:00:33 --> 00:00:34
script that was easy available.

8
00:00:35 --> 00:00:37
You had to do a couple of steps to do that.

9
00:00:37 --> 00:00:43
But thanks to a very kind soul we don't have the name of the user so it's pluys.

10
00:00:45 --> 00:00:52
So pluys has actually created a helper script that will help you run Facebook's NLLB

11
00:00:52 --